## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 1
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_1


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [6]:
X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Las Vegas Raiders,1,2021,33.0,7.8,409.0,89.5,12.8,82.0,27.0,...,12.4,189.0,18.0,15.0,12.0,2.0,0.882353,0.05,2.0,2.0
1,Cleveland Browns,1,2021,29.0,11.5,304.0,97.5,15.3,153.0,33.0,...,12.5,73.0,13.0,7.0,13.0,0.0,1.565217,0.00,0.0,0.0


In [7]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [8]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
10,Arizona Cardinals,2021,week_1,0.373532,0.451286,0.692689,0.532364,0.512468
16,Los Angeles Rams,2021,week_1,0.351888,0.409373,0.649777,0.535625,0.486665
4,San Francisco 49ers,2021,week_1,0.328197,0.327513,0.699347,0.455746,0.452701


In [9]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [10]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
10,Arizona Cardinals,2021,week_1,0.373532,0.451286,0.692689,0.532364,0.512468
16,Los Angeles Rams,2021,week_1,0.351888,0.409373,0.649777,0.535625,0.486665
4,San Francisco 49ers,2021,week_1,0.328197,0.327513,0.699347,0.455746,0.452701
17,Seattle Seahawks,2021,week_1,0.309283,0.326143,0.531376,0.371959,0.384690
5,Philadelphia Eagles,2021,week_1,0.282938,0.252232,0.376934,0.357785,0.317472
6,Tampa Bay Buccaneers,2021,week_1,0.239050,0.177787,0.480186,0.285018,0.295510
23,New Orleans Saints,2021,week_1,0.266191,0.227318,0.288094,0.360903,0.285627
20,Cincinnati Bengals,2021,week_1,0.219908,0.137199,0.470293,0.292767,0.280042
13,Kansas City Chiefs,2021,week_1,0.262862,0.201902,0.242276,0.276996,0.246009
3,Houston Texans,2021,week_1,0.256274,0.194047,0.218999,0.271492,0.235203
